In [1]:
import pandas as pd
from io import StringIO
import os

## Read trace

In [2]:
def read_trace(trace_dir, trace_file, worker_id=None):
    trace_filepath = os.path.join(trace_dir, trace_file)
    
    with open(trace_filepath,"r") as rf:
        new_trace = StringIO()
        line = rf.readline()
        while line:
            line_list = line.split()
            if line_list[0] == "[trace]":
                new_trace.write(line)
            line = rf.readline()
       
    new_trace.seek(0)
    df = pd.read_csv(new_trace, sep=" ", names=["InfoType", "EventType", "EventState", "ID", "Time"])
    del df["InfoType"]
    
    if worker_id is not None:
        df.insert(0, "WorkerID", worker_id)
        
    new_trace.close()
    
    return df

### COORD

In [3]:
trace_dir = "/home/posheng/billy_ws/fl_025/paslab/trace/async2_fl_bw_3rm/100mbps"
coord_trace_file = "coord.txt"
hpc0_trace_file = "hpc0.txt"
hpc1_trace_file = "hpc1.txt"
hpc2_trace_file = "hpc2.txt"
# hpc3_trace_file = "hpc3_trace.txt"

In [4]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

coord_df = read_trace(trace_dir, coord_trace_file)
w0_df = read_trace(trace_dir, hpc0_trace_file, "alice")
w1_df = read_trace(trace_dir, hpc1_trace_file, "bob")
w2_df = read_trace(trace_dir, hpc2_trace_file, "charlie")

In [5]:
hpc1_df = w0_df
hpc2_df = w1_df
hpc3_df = w2_df

In [6]:
worker_dfs = {
    "alice": hpc1_df,
    "bob": hpc2_df,
    "charlie": hpc3_df
}

## Duration

In [7]:
def get_worker_duration(worker_id, worker_dfs, event_type):
    worker_df = worker_dfs[worker_id]
    worker_event_df = worker_df[worker_df["EventType"].str.contains(event_type, regex=False)]
    worker_event_df = worker_event_df.reset_index()
    
    del worker_event_df['index']
    del worker_event_df['ID']
    
    return worker_event_df

In [8]:
def get_coord_duration(coord_df, event_type):
    coord_event_df = coord_df[coord_df["EventType"].str.contains(event_type, regex=False)]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    del coord_event_df['ID']
    
    return coord_event_df

In [9]:
def calculate_duration(event_type, df):
    event_df = df[df["EventType"] == event_type][["EventType", "EventState", "ID", "Time"]]
    event_start_df = event_df[event_df["EventState"] == "start"][["ID", "Time"]]
    event_start_df = event_start_df.reset_index()

    event_end_df = event_df[event_df["EventState"] == "end"][["Time"]]
    event_end_df = event_end_df.reset_index()
    
    event_df_cat = pd.concat([event_start_df, event_end_df], axis=1)
    del event_df_cat['index']
    event_df_cat.columns = ["ID", "StartTime", "EndTime"]
    
    event_df_cat["Duration"] = event_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    event_df_cat.insert(0, "EventType", event_type)
    return event_df_cat


In [10]:
GlobalModelSendDuration = calculate_duration("GlobalModelSend", coord_df)

In [11]:
LossFuncSendDuration = calculate_duration("LossFuncSend", coord_df)

In [12]:
ModelConfigSendDuration = calculate_duration("ModelConfigSend", coord_df)

### GlobalModelSendDuration

In [13]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "alice"][1:]['Duration'].mean()

0.7651961922645569

In [14]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "bob"][1:]['Duration'].mean()

0.7568269848823548

In [15]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

0.7636773109436035

### LossFuncSendDuration

In [16]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "alice"][1:]['Duration'].mean()

0.0014116764068603516

In [17]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "bob"][1:]['Duration'].mean()

0.001397240161895752

In [18]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

0.0013011336326599122

### ModelConfigSendDuration

In [19]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "alice"][1:]['Duration'].mean()

0.0010106205940246583

In [20]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "bob"][1:]['Duration'].mean()

0.0010421037673950194

In [21]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

0.0009946823120117188

## GlobalInformationSend Duration

In [22]:
def get_coord_global_send_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("GlobalInformationSend", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [42]:
1602055807.1283371 - 1602055807.1122975

0.016039609909057617

In [23]:
hpc1_gs_df = get_coord_global_send_duration(coord_df, "alice")

In [24]:
hpc2_gs_df = get_coord_global_send_duration(coord_df, "bob")

In [25]:
hpc3_gs_df = get_coord_global_send_duration(coord_df, "charlie")

In [30]:
hpc1_gs_df['Time']

0     0.768052
1     0.778790
2     0.749535
3     0.753522
4     0.783623
5     0.687530
6     0.724591
7     0.790412
8     0.706375
9     0.795602
10    0.777714
11    0.763020
12    0.791592
13    0.790056
14    0.788557
15    0.772515
16    0.786885
17    0.755273
18    0.740917
19    0.778038
20    0.788069
Name: Time, dtype: float64

In [26]:
hpc1_gs_df['Time'][1:].mean()

0.7651306986808777

In [27]:
hpc2_gs_df['Time'][1:].mean()

0.7618660926818848

In [28]:
hpc3_gs_df['Time'][1:].mean()

0.7659916877746582

## Send Time

In [29]:
## EventType:
##     * GlobalModelSend
##     * LossFuncSend
##     * ModelConfigSend
def calculate_send_time(coord_df, worker_dfs, event_type):
    event_start_df = coord_df[(coord_df["EventType"] == event_type) & (coord_df["EventState"] == "start")]
    worker_event_send_time_dfs = {}
    
    for worker_id in worker_dfs.keys():
        worker_event_start_df = event_start_df[event_start_df["ID"] == worker_id]
        worker_event_start_df = worker_event_start_df.reset_index()
        del worker_event_start_df['index']
        worker_event_start_df.columns = ["EventType", "EventState", "ID", "StartTime"]
        
        worker_df = worker_dfs[worker_id]
        worker_event_recv_df = worker_df[worker_df["EventType"] == event_type]
        worker_event_recv_df = worker_event_recv_df.reset_index()
        del worker_event_recv_df['index']
        worker_event_recv_df.columns = ["WorkerID", "EventType", "EventState", "ID", "RecvTime"]
        
        worker_recv_time_series = worker_event_recv_df["RecvTime"]
        worker_event_send_time_df = pd.concat([worker_event_start_df, worker_recv_time_series], axis=1)
        worker_event_send_time_df["SendTime"] = worker_event_send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
        del worker_event_send_time_df["EventState"]
        
        worker_event_send_time_dfs[worker_id] = worker_event_send_time_df
        
    return worker_event_send_time_dfs

In [30]:
worker_gms_send_time_dfs = calculate_send_time(coord_df, worker_dfs, "GlobalModelSend")

In [31]:
worker_gms_send_time_dfs['alice']['SendTime'].mean()

0.7485054901668003

In [32]:
worker_gms_send_time_dfs['alice']

,EventType,ID,StartTime,RecvTime,SendTime
0,GlobalModelSend,alice,1.601986e+09,1.601986e+09,0.751460
1,GlobalModelSend,alice,1.601986e+09,1.601986e+09,0.762440
2,GlobalModelSend,alice,1.601986e+09,1.601986e+09,0.732629
3,GlobalModelSend,alice,1.601986e+09,1.601986e+09,0.736666
4,GlobalModelSend,alice,1.601986e+09,1.601986e+09,0.765949
5,GlobalModelSend,alice,1.601986e+09,1.601986e+09,0.670759
6,GlobalModelSend,alice,1.601986e+09,1.601986e+09,0.707843
7,GlobalModelSend,alice,1.601987e+09,1.601987e+09,0.772914
8,GlobalModelSend,alice,1.601987e+09,1.601987e+09,0.689674
9,GlobalModelSend,alice,1.601987e+09,1.601987e+09,0.778400


In [33]:
worker_gms_send_time_dfs['charlie']['SendTime'].mean()

0.7464504241943359

In [34]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

## LocalTraining Duration

In [35]:
hpc1_lt_df = get_worker_duration("alice", worker_dfs, "LocalTraining")

In [36]:
hpc2_lt_df = get_worker_duration("bob", worker_dfs, "LocalTraining")

In [37]:
hpc3_lt_df = get_worker_duration("charlie", worker_dfs, "LocalTraining")

In [38]:
hpc1_lt_df['Time'][1:].mean()

13.869032680988312

In [39]:
hpc2_lt_df['Time'][1:].mean()

9.491367208957673

In [40]:
hpc3_lt_df['Time'][1:].mean()

10.143671476840973

In [41]:
hpc1_lt_df

,WorkerID,EventType,EventState,Time
0,alice,LocalTraining,duration,13.992475
1,alice,LocalTraining,duration,13.618030
2,alice,LocalTraining,duration,14.094082
3,alice,LocalTraining,duration,14.257969
4,alice,LocalTraining,duration,13.879264
5,alice,LocalTraining,duration,13.661154
6,alice,LocalTraining,duration,14.114460
7,alice,LocalTraining,duration,13.346154
8,alice,LocalTraining,duration,13.654593
9,alice,LocalTraining,duration,14.113661


## FitSagg Duration

In [42]:
def get_coord_fitsagg_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("FitSagg", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [43]:
hpc1_fs_df = get_coord_fitsagg_duration(coord_df, "alice")

In [44]:
hpc2_fs_df = get_coord_fitsagg_duration(coord_df, "bob")

In [45]:
hpc3_fs_df = get_coord_fitsagg_duration(coord_df, "charlie")

In [46]:
hpc1_fs_df['Time'][1:].mean()

17.528527224063872

In [47]:
hpc2_fs_df['Time'][1:].mean()

17.610635459423065

In [48]:
hpc3_fs_df['Time'][1:].mean()

17.66348727941513

## EncryptMultiply Duration

In [49]:
hpc1_em_df = get_worker_duration("alice", worker_dfs, "EncryptMultiply")

In [50]:
hpc2_em_df = get_worker_duration("bob", worker_dfs, "EncryptMultiply")

In [51]:
hpc3_em_df = get_worker_duration("charlie", worker_dfs, "EncryptMultiply")

In [52]:
hpc1_em_df['Time'][1:].mean()

2.8525857806205748

In [101]:
hpc2_em_df['Time'][1:].mean()

7.318153047561646

In [102]:
hpc3_em_df['Time'][1:].mean()

6.714117050170898

In [103]:
hpc1_em_df

,WorkerID,EventType,EventState,Time
0,alice,EncryptMultiply,duration,3.278606
1,alice,EncryptMultiply,duration,2.731663
2,alice,EncryptMultiply,duration,2.714762
3,alice,EncryptMultiply,duration,2.770517
4,alice,EncryptMultiply,duration,3.321286
5,alice,EncryptMultiply,duration,2.538695
6,alice,EncryptMultiply,duration,2.489272
7,alice,EncryptMultiply,duration,3.117352
8,alice,EncryptMultiply,duration,2.836988
9,alice,EncryptMultiply,duration,3.107404


In [104]:
hpc3_em_df

,WorkerID,EventType,EventState,Time
0,charlie,EncryptMultiply,duration,7.060598
1,charlie,EncryptMultiply,duration,7.274583
2,charlie,EncryptMultiply,duration,7.501222
3,charlie,EncryptMultiply,duration,7.464529
4,charlie,EncryptMultiply,duration,7.319272
5,charlie,EncryptMultiply,duration,6.121235
6,charlie,EncryptMultiply,duration,6.372397
7,charlie,EncryptMultiply,duration,6.816015
8,charlie,EncryptMultiply,duration,7.042194
9,charlie,EncryptMultiply,duration,6.717224


## EncryptParameter Duration

In [105]:
def get_encrypt_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('EncryptParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df
    

In [106]:
# hpc1_ep_df = get_encrypt_parameter_duration("alice", worker_dfs)
hpc1_ep_df = get_worker_duration("alice", worker_dfs, "EncryptParameter")
hpc2_ep_df = get_worker_duration("bob", worker_dfs, "EncryptParameter")
hpc3_ep_df = get_worker_duration("charlie", worker_dfs, "EncryptParameter")

In [107]:
hpc1_ep_df[0:8]

,WorkerID,EventType,EventState,Time
0,alice,EncryptParameter0,duration,0.008179
1,alice,EncryptParameter1,duration,0.005694
2,alice,EncryptParameter2,duration,0.174992
3,alice,EncryptParameter3,duration,0.051831
4,alice,EncryptParameter4,duration,2.885745
5,alice,EncryptParameter5,duration,0.015123
6,alice,EncryptParameter6,duration,0.035656
7,alice,EncryptParameter7,duration,0.004223


In [108]:
mean_values = []
for i in range(8):
    mean_value = hpc1_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    mean_values.append(mean_value)
    print(str(i)+":", mean_value)

0: 0.008680979410807291
1: 0.005238044829595657
2: 0.14933973266964867
3: 0.024828740528651645
4: 2.4432803222111295
5: 0.010082619530814034
6: 0.0348263354528518
7: 0.007713851474580311


In [109]:
sum(mean_values)

2.6839906261080793

In [110]:
for i in range(8):
    mean_value = hpc2_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    print(str(i)+":", mean_value)

0: 4.412874323981149
1: 0.004431883494059245
2: 0.17683457192920504
3: 0.020685547874087377
4: 2.4393782956259593
5: 0.009996039526803153
6: 0.07936409541538783
7: 0.05307532492138091


In [111]:
for i in range(8):
    mean_value = hpc3_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    print(str(i)+":", mean_value)

0: 3.738501094636463
1: 0.004783607664562407
2: 0.16044934590657553
3: 0.0194485414595831
4: 2.589001258214315
5: 0.008306877953665597
6: 0.03576522781735375
7: 0.006612834476289295


In [112]:
hpc1_ep_df["Time"][0:8]

0    0.008179
1    0.005694
2    0.174992
3    0.051831
4    2.885745
5    0.015123
6    0.035656
7    0.004223
Name: Time, dtype: float64

In [113]:
hpc1_ep_df["Time"].mean()

0.33549882826350985

In [114]:
hpc2_ep_df["Time"].mean()

0.899580010346004

In [115]:
hpc3_ep_df["Time"].mean()

0.8203585985161009

## MultiplyParameter Duration

In [116]:
def get_multiply_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('MultiplyParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df

In [117]:
# hpc1_mp_df = get_multiply_parameter_duration("alice", worker_dfs)
hpc1_mp_df = get_worker_duration("alice", worker_dfs, "MultiplyParameter")
hpc2_mp_df = get_worker_duration("bob", worker_dfs, "MultiplyParameter")
hpc3_mp_df = get_worker_duration("charlie", worker_dfs, "MultiplyParameter")

In [118]:
hpc1_mp_df[:8]["Time"].sum()

0.0924208164215088

In [119]:
hpc1_mp_df["Time"].mean()

0.02310183218547276

In [120]:
hpc2_mp_df["Time"].mean()

0.018719817910875593

In [121]:
hpc3_mp_df["Time"].mean()

0.020499879405612036

## RetrieveTime duration

In [122]:
def get_coord_retrieve_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("RetrieveTime", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [123]:
hpc1_ret_df = get_coord_retrieve_duration(coord_df, "alice")

In [124]:
hpc2_ret_df = get_coord_retrieve_duration(coord_df, "bob")

In [125]:
hpc3_ret_df = get_coord_retrieve_duration(coord_df, "charlie")

In [126]:
hpc1_ret_df["Time"][1:].mean()

0.017100870609283447

In [127]:
hpc2_ret_df["Time"][1:].mean()

0.015381324291229247

In [128]:
hpc3_ret_df["Time"][1:].mean()

0.01484755277633667

## ShareSend Duration

In [129]:
def calculate_share_send_duration(worker_id, worker_dfs, nr_parameters):
    worker_df = worker_dfs[worker_id]
    worker_send_share_df = worker_df[worker_df["EventType"].str.contains('SendShare', regex=False)]
    
    worker_send_share_start_df = worker_send_share_df[worker_send_share_df["EventState"] == 'start']
    worker_send_share_start_df = worker_send_share_start_df.reset_index()
    worker_send_share_start_df = worker_send_share_start_df[["WorkerID", "EventType", "ID", "Time"]]
    worker_send_share_start_df.columns = ["WorkerID", "EventType", "ID", "StartTime"]
    
    worker_send_share_end_df = worker_send_share_df[worker_send_share_df["EventState"] == 'end']
    worker_send_share_end_df = worker_send_share_end_df.reset_index()
    
    send_share_df_cat = pd.concat([worker_send_share_start_df, worker_send_share_end_df["Time"]], axis=1)
    send_share_df_cat.columns = ["WorkerID", "EventType", "ID", "StartTime", "EndTime"]
    send_share_df_cat["Duration"] = send_share_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    
    parameter_index = []
    nr_worker = len(worker_dfs)
    p_index = -1

    for i in range(len(send_share_df_cat)):
        tmp_index = (i%nr_worker)
        if tmp_index == 0:
            p_index += 1
            p_index = p_index % nr_parameters
        parameter_index.append(p_index)
        
    send_share_df_cat.insert(2, "p_idx", parameter_index)
    obj_id_list = send_share_df_cat.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_share_df_cat.insert(2, "obj_id", obj_id_list)
    
    del send_share_df_cat["EventType"]
    send_share_df_cat.insert(1, "EventType", "SendShare")
    
    return send_share_df_cat


In [130]:
alice_send_share_df_cat = calculate_share_send_duration("alice", worker_dfs, 8)
bob_send_share_df_cat = calculate_share_send_duration("bob", worker_dfs, 8)
charlie_send_share_df_cat = calculate_share_send_duration("charlie", worker_dfs, 8)

In [131]:
alice_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,alice,SendShare,94631695399,0,alice,1.601986e+09,1.601986e+09,0.001004
1,alice,SendShare,51867600145,0,bob,1.601986e+09,1.601986e+09,0.003412
2,alice,SendShare,99017192652,0,charlie,1.601986e+09,1.601986e+09,0.003241
3,alice,SendShare,39209828862,1,alice,1.601986e+09,1.601986e+09,0.000537
4,alice,SendShare,2805691438,1,bob,1.601986e+09,1.601986e+09,0.003325
...,...,...,...,...,...,...,...,...
499,alice,SendShare,75768864211,6,bob,1.601987e+09,1.601987e+09,0.024134
500,alice,SendShare,45632418653,6,charlie,1.601987e+09,1.601987e+09,0.014672
501,alice,SendShare,74115743881,7,alice,1.601987e+09,1.601987e+09,0.000913
502,alice,SendShare,32437666830,7,bob,1.601987e+09,1.601987e+09,0.007008


In [132]:
bob_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,bob,SendShare,71753566022,0,alice,1.601986e+09,1.601986e+09,4.754788
1,bob,SendShare,33316160301,0,bob,1.601986e+09,1.601986e+09,0.001015
2,bob,SendShare,70698033332,0,charlie,1.601986e+09,1.601986e+09,0.004194
3,bob,SendShare,30564881151,1,alice,1.601986e+09,1.601986e+09,0.001814
4,bob,SendShare,15935525561,1,bob,1.601986e+09,1.601986e+09,0.000603
...,...,...,...,...,...,...,...,...
499,bob,SendShare,23196229533,6,bob,1.601987e+09,1.601987e+09,0.000677
500,bob,SendShare,7419015346,6,charlie,1.601987e+09,1.601987e+09,0.015789
501,bob,SendShare,27199016963,7,alice,1.601987e+09,1.601987e+09,0.001944
502,bob,SendShare,64741868678,7,bob,1.601987e+09,1.601987e+09,0.000951


In [133]:
charlie_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,charlie,SendShare,74268282113,0,alice,1.601986e+09,1.601986e+09,3.793310
1,charlie,SendShare,33722881246,0,bob,1.601986e+09,1.601986e+09,0.006335
2,charlie,SendShare,21928928843,0,charlie,1.601986e+09,1.601986e+09,0.000960
3,charlie,SendShare,31942521795,1,alice,1.601986e+09,1.601986e+09,0.001678
4,charlie,SendShare,11292091875,1,bob,1.601986e+09,1.601986e+09,0.001790
...,...,...,...,...,...,...,...,...
499,charlie,SendShare,86031795622,6,bob,1.601987e+09,1.601987e+09,0.022573
500,charlie,SendShare,87419627585,6,charlie,1.601987e+09,1.601987e+09,0.000463
501,charlie,SendShare,17959139505,7,alice,1.601987e+09,1.601987e+09,0.001504
502,charlie,SendShare,52096714640,7,bob,1.601987e+09,1.601987e+09,0.003509


## ShareSend Send Time

In [134]:
def calculate_share_send_time(sender_id, receiver_id, worker_dfs, nr_parameter):
    sender_df = worker_dfs[sender_id]
    receiver_df = worker_dfs[receiver_id]
    
    sender_send_share_df = sender_df[sender_df["EventType"].str.contains('SendShare', regex=False)]
    receiver_get_share_df = receiver_df[receiver_df["EventType"].str.contains('GetShare', regex=False)]
    
    send_share_start_df = sender_send_share_df[(sender_send_share_df["ID"] == receiver_id) & (sender_send_share_df["EventState"] == 'start')]
    get_share_df = receiver_get_share_df[receiver_get_share_df["ID"] == sender_id]
    send_share_start_df = send_share_start_df.reset_index()
    get_share_df = get_share_df.reset_index()
    
    del send_share_start_df['index']
    del send_share_start_df['EventState']
    
    send_time_df = pd.concat([send_share_start_df, get_share_df["Time"]], axis=1)
    send_time_df.columns = ["WorkerID", "EventType", "ID", "StartTime", "RecvTime"]
    send_time_df["SendTime"] = send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
    
    nr_parameter = 8
    parameter_index = []

    for i in range(len(send_time_df)):
        tmp_index = (i%nr_parameter)
        parameter_index.append(tmp_index)
        
    send_time_df.insert(2, "p_idx", parameter_index)
    
    obj_id_list = send_time_df.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_time_df.insert(2, "obj_id", obj_id_list)
    
    del send_time_df["EventType"]
    send_time_df.insert(1, "EventType", "SendShare")
    
    return send_time_df

In [135]:
send_time_df = calculate_share_send_time("alice", "bob", worker_dfs, 8)

In [136]:
send_time_df

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,RecvTime,SendTime
0,alice,SendShare,51867600145,0,bob,1.601986e+09,1.601986e+09,0.002847
1,alice,SendShare,2805691438,1,bob,1.601986e+09,1.601986e+09,0.002742
2,alice,SendShare,84852166780,2,bob,1.601986e+09,1.601986e+09,0.060992
3,alice,SendShare,5603916406,3,bob,1.601986e+09,1.601986e+09,0.045604
4,alice,SendShare,38391999345,4,bob,1.601986e+09,1.601986e+09,1.486456
...,...,...,...,...,...,...,...,...
163,alice,SendShare,42879180074,3,bob,1.601987e+09,1.601987e+09,0.018521
164,alice,SendShare,41458084762,4,bob,1.601987e+09,1.601987e+09,0.920157
165,alice,SendShare,34103653186,5,bob,1.601987e+09,1.601987e+09,0.002619
166,alice,SendShare,75768864211,6,bob,1.601987e+09,1.601987e+09,0.020775


## AllWorkersTrainingTime Duration

In [137]:
coord_awtt_df = get_coord_duration(coord_df, "AllWorkersTrainingTime")

In [138]:
coord_awtt_df['Time'][1:].mean()

17.94358605146408

In [139]:
coord_awtt_df

,EventType,EventState,Time
0,AllWorkersTrainingTime,duration,18.088955
1,AllWorkersTrainingTime,duration,17.760969
2,AllWorkersTrainingTime,duration,18.355916
3,AllWorkersTrainingTime,duration,18.476558
4,AllWorkersTrainingTime,duration,18.067665
5,AllWorkersTrainingTime,duration,17.527401
6,AllWorkersTrainingTime,duration,18.064381
7,AllWorkersTrainingTime,duration,17.344831
8,AllWorkersTrainingTime,duration,17.913636
9,AllWorkersTrainingTime,duration,18.107202


## AggregationTime

In [140]:
coord_aggt_df = get_coord_duration(coord_df, "AggregationTime")

In [141]:
coord_aggt_df["Time"][1:].mean()

0.1080413818359375

In [142]:
coord_aggt_df

,EventType,EventState,Time
0,AggregationTime,duration,0.105232
1,AggregationTime,duration,0.112568
2,AggregationTime,duration,0.109736
3,AggregationTime,duration,0.108742
4,AggregationTime,duration,0.109940
5,AggregationTime,duration,0.105813
6,AggregationTime,duration,0.107059
7,AggregationTime,duration,0.107715
8,AggregationTime,duration,0.104943
9,AggregationTime,duration,0.107641


## DecryptionTime

In [143]:
coord_decrypt_df = get_coord_duration(coord_df, "DecryptionTime")

In [144]:
coord_decrypt_df['Time'][1:].mean()

0.9501947045326233

In [145]:
coord_decrypt_df

,EventType,EventState,Time
0,DecryptionTime,duration,0.951632
1,DecryptionTime,duration,0.957067
2,DecryptionTime,duration,0.958531
3,DecryptionTime,duration,0.952799
4,DecryptionTime,duration,0.954839
5,DecryptionTime,duration,0.961585
6,DecryptionTime,duration,0.948813
7,DecryptionTime,duration,0.957397
8,DecryptionTime,duration,0.950274
9,DecryptionTime,duration,0.946351


## RoundTime

In [146]:
coord_round_time_df = get_coord_duration(coord_df, "RoundTime")

In [147]:
coord_round_time_df["Time"][1:].mean()

19.043557941913605

In [100]:
coord_round_time_df

,EventType,EventState,Time
0,RoundTime,duration,19.145947
1,RoundTime,duration,18.873726
2,RoundTime,duration,19.465924
3,RoundTime,duration,19.579201
4,RoundTime,duration,19.174881
5,RoundTime,duration,18.636325
6,RoundTime,duration,19.162090
7,RoundTime,duration,18.452327
8,RoundTime,duration,19.007770
9,RoundTime,duration,19.201967
